In [3]:
#!pip install shap optuna
#from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
import shap

import optuna
from sklearn.model_selection import train_test_split
from optuna.integration import LightGBMPruningCallback
from google.colab import files

#Reading the csv data into a variable. CSV was stored on Google Drive.
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/train.csv")

#Removing the non-number columns from df. Storing all but last columns in x, last column in y
df_int = df._get_numeric_data()
x=df_int.drop("SalePrice",axis=1)
y=df_int.SalePrice
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

#Tuning the hyperparameters with Optuna
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        'metric': 'rmse',
        "n_estimators": trial.suggest_categorical("n_estimators", [20000]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 10, 100),
        #"min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.9, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.9, step=0.1
        ),
        #'min_child_samples': trial.suggest_int('min_child_samples', 1, 150),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    olgbm = lgbm.LGBMRegressor(**param)
    olgbm.fit(X_train, y_train,eval_set=[(X_test,y_test)], callbacks=[lgbm.early_stopping(100), lgbm.log_evaluation, LightGBMPruningCallback(trial,"rmse")])
    predictions = olgbm.predict(X_test)
    accuracy = mean_squared_error(y_test, predictions)
    return accuracy

study = optuna.create_study(study_name="lightgbm", direction="minimize")
study.optimize(objective, n_trials=50)

model = lgbm.LGBMRegressor(**study.best_params)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
#model.booster_.save_model("OptimizedModel.txt")
#files.download("OptimizedModel.txt")

print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

fig1 = optuna.visualization.plot_optimization_history(study)
fig2 = optuna.visualization.plot_slice(study)
fig3 = optuna.visualization.plot_param_importances(study)
fig1.show()
fig2.show()
fig3.show()

for i in range(len(prediction)):
  print("Optuna Price: {}".format(prediction[i]))

[I 2023-04-17 03:04:46,888] A new study created in memory with name: lightgbm


[LightGBM] [Warning] lambda_l1 is set=50, reg_alpha=0.0 will be ignored. Current value: lambda_l1=50
[LightGBM] [Warning] min_gain_to_split is set=8.532899149230442, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=8.532899149230442
[LightGBM] [Warning] lambda_l2 is set=75, reg_lambda=0.0 will be ignored. Current value: lambda_l2=75
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=0.6 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 100 rounds


[I 2023-04-17 03:04:48,469] Trial 0 finished with value: 850025325.7283984 and parameters: {'n_estimators': 20000, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.1029805430313656, 'num_leaves': 1760, 'max_depth': 23, 'lambda_l1': 50, 'lambda_l2': 75, 'min_gain_to_split': 8.532899149230442, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.7, 'min_data_per_groups': 74}. Best is trial 0 with value: 850025325.7283984.


Early stopping, best iteration is:
[680]	valid_0's rmse: 29155.2
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] min_gain_to_split is set=4.051592564686907, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.051592564686907
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 100 rounds


[I 2023-04-17 03:04:51,837] Trial 1 finished with value: 808212359.0774509 and parameters: {'n_estimators': 20000, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.16926232578759773, 'num_leaves': 2800, 'max_depth': 59, 'lambda_l1': 5, 'lambda_l2': 0, 'min_gain_to_split': 4.051592564686907, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.5, 'min_data_per_groups': 39}. Best is trial 1 with value: 808212359.0774509.


...

[I 2023-04-17 03:05:11,006] Trial 47 finished with value: 849572619.6934067 and parameters: {'n_estimators': 20000, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.23975455867196552, 'num_leaves': 1780, 'max_depth': 54, 'lambda_l1': 75, 'lambda_l2': 10, 'min_gain_to_split': 2.9288248148567515, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.7, 'min_data_per_groups': 70}. Best is trial 43 with value: 777931457.2987448.
[I 2023-04-17 03:05:11,108] Trial 48 pruned. Trial was pruned at iteration 0.


Early stopping, best iteration is:
[98]	valid_0's rmse: 29147.4
[LightGBM] [Warning] lambda_l1 is set=65, reg_alpha=0.0 will be ignored. Current value: lambda_l1=65
[LightGBM] [Warning] min_gain_to_split is set=5.226829291920923, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.226829291920923
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=0.8 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-04-17 03:05:11,214] Trial 49 pruned. Trial was pruned at iteration 0.


	Best value (rmse): 777931457.29874
	Best params:
		n_estimators: 20000
		colsample_bytree: 0.7
		subsample: 0.6
		learning_rate: 0.247984272795805
		num_leaves: 2480
		max_depth: 24
		lambda_l1: 75
		lambda_l2: 0
		min_gain_to_split: 3.160242356640873
		bagging_fraction: 0.9
		bagging_freq: 1
		feature_fraction: 0.7
		min_data_per_groups: 69


Optuna Price: 225337.30034241217
Optuna Price: 182019.6523214714
Optuna Price: 111352.21606774292
Optuna Price: 66325.56311194341
Optuna Price: 151272.80504644866
Optuna Price: 346979.96954410255
Optuna Price: 315074.5425273202
Optuna Price: 146623.18295361853
Optuna Price: 215871.43893933244
Optuna Price: 215912.4565592424
Optuna Price: 179227.20201527252
Optuna Price: 80636.76889063501
Optuna Price: 183350.28622159772
Optuna Price: 316231.5072902837
Optuna Price: 243178.67528713957
Optuna Price: 109117.04577070524
Optuna Price: 110324.31948691314
Optuna Price: 113779.51182541635
Optuna Price: 238683.56555092987
Optuna Price: 119952.39492711765
Optuna Price: 118924.975305371
Optuna Price: 123200.95857740453
Optuna Price: 275322.7634276986
Optuna Price: 307252.02703249076
Optuna Price: 102577.61861432229
Optuna Price: 218139.89452493846
Optuna Price: 139251.53452757892
Optuna Price: 175445.78364563372
Optuna Price: 515635.4439715337
Optuna Price: 138877.07914641002
Optuna Price: 111410